In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Input
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import random
from matplotlib import pyplot as plt
from PIL import Image
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import Callback

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
folder1 = './skin-cancer-mnist-ham10000/HAM10000_images_part_1'
folder2 = './skin-cancer-mnist-ham10000/HAM10000_images_part_2'

images1 = [os.path.join(folder1, f) for f in os.listdir(folder1) if f.endswith('.jpg')]
images2 = [os.path.join(folder2, f) for f in os.listdir(folder2) if f.endswith('.jpg')]

random_images1 = random.sample(images1, 5)
random_images2 = random.sample(images2, 5)

def display_images(images, title):
    plt.figure(figsize=(15, 3))  #Ajustar el tamaño de la figura.
    for i, img_path in enumerate(images):
        img = Image.open(img_path)
        plt.subplot(1, 5, i + 1)  # Crear un subplot para cada imagen
        plt.imshow(img)
        plt.axis('off')  # No mostrar los ejes
        plt.title(f"Imagen {i + 1}")
    plt.suptitle(title)
    plt.show()

display_images(random_images1, "5 imagenes aletorias de la carpeta 1")
display_images(random_images2, "5 imagenes aletorias de la carpeta 2")

In [ ]:
tabular_data = pd.read_csv('./skin-cancer-mnist-ham10000/HAM10000_metadata.csv')

print(tabular_data.head())

In [ ]:
data = pd.read_csv('./skin-cancer-mnist-ham10000/hmnist_28_28_RGB.csv')

In [ ]:
x = data.drop('label', axis=1)
y = data['label']

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# Configurar semillas para reproducibilidad
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Función para balancear datos usando sklearn (alternativa a imblearn)
def balance_data(x, y, random_state=SEED):
    """Balancea los datos usando oversampling"""
    # Convertir a numpy si es pandas DataFrame
    if hasattr(x, 'values'):
        x_array = x.values
    else:
        x_array = np.array(x)
    
    if hasattr(y, 'values'):
        y_array = y.values
    else:
        y_array = np.array(y)
    
    unique_classes, counts = np.unique(y_array, return_counts=True)
    max_count = max(counts)
    
    x_balanced = []
    y_balanced = []
    
    for class_label in unique_classes:
        class_indices = np.where(y_array == class_label)[0]
        class_x = x_array[class_indices]
        class_y = y_array[class_indices]
        
        # Oversample para igualar a la clase mayoritaria
        if len(class_x) < max_count:
            class_x_resampled, class_y_resampled = resample(
                class_x, class_y, 
                n_samples=max_count, 
                random_state=random_state
            )
        else:
            class_x_resampled, class_y_resampled = class_x, class_y
            
        x_balanced.append(class_x_resampled)
        y_balanced.append(class_y_resampled)
    
    return np.vstack(x_balanced), np.hstack(y_balanced)

# Aplicar balanceado de datos
x_resampled, y_resampled = balance_data(x, y)
x_resampled = np.array(x_resampled).reshape(-1, 28, 28, 3).astype(np.float32)

# Normalización
mean = np.mean(x_resampled)
std = np.std(x_resampled)
x_resampled = (x_resampled - mean) / std

# Seleccionar muestras para augmentation
num_augmented_samples = int(0.005 * len(x_resampled))
rng = np.random.default_rng(SEED)
indices = rng.choice(len(x_resampled), num_augmented_samples, replace=False)
x_selected = x_resampled[indices]
y_selected = y_resampled[indices]

# Data Augmentation usando TensorFlow
def augment_images(images):
    """Aplica data augmentation usando TensorFlow"""
    augmented_images = []
    
    for img in images:
        img = tf.expand_dims(img, 0)
        
        if tf.random.uniform([]) > 0.5:
            img = tf.image.flip_left_right(img)
        
        if tf.random.uniform([]) > 0.5:
            img = tf.image.flip_up_down(img)
        
        if tf.random.uniform([]) > 0.5:
            img = tf.image.rot90(img, k=tf.random.uniform([], minval=0, maxval=4, dtype=tf.int32))
        
        img = tf.image.random_brightness(img, max_delta=0.1)
        img = tf.image.random_contrast(img, lower=0.9, upper=1.1)
        img = tf.image.random_saturation(img, lower=0.9, upper=1.1)
        
        noise = tf.random.normal(shape=tf.shape(img), mean=0.0, stddev=0.01)
        img = img + noise
        
        img = tf.image.random_crop(img, size=[1, 25, 25, 3])
        img = tf.image.resize(img, [28, 28])
        img = tf.clip_by_value(img, -3.0, 3.0)
        
        augmented_images.append(tf.squeeze(img, 0))
    
    return tf.stack(augmented_images)

# Aplicar data augmentation
x_augmented = augment_images(x_selected)

# Combinar datos originales y aumentados
x_resampled = np.concatenate((x_resampled, x_augmented.numpy()), axis=0)
y_resampled = np.concatenate((y_resampled, y_selected), axis=0)

# División en entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(
    x_resampled, y_resampled, test_size=0.2, random_state=SEED, stratify=y_resampled
)

print(f'Training set size: {X_train.shape}, Testing set size: {X_test.shape}')

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import Callback

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
rng = np.random.default_rng(SEED)

class CustomLearningRateScheduler(Callback):
    def __init__(self, h=1.0, m_range=10, patience=3, random_factor=0.1, decay_rate=0.98, warmup_epochs=5, min_lr=1e-6, max_adjustment=0.0005, verbose=0):
        super(CustomLearningRateScheduler, self).__init__()
        self.h = h
        self.m_range = m_range
        self.patience = patience
        self.random_factor = random_factor
        self.decay_rate = decay_rate
        self.warmup_epochs = warmup_epochs
        self.min_lr = min_lr
        self.max_adjustment = max_adjustment
        self.verbose = verbose
        self.best_loss = float('inf')
        self.loss_history = []

    def compute_adjustment_factor(self, logs):
        current_loss = logs.get("loss", 1.0)
        g = lambda x: tf.nn.relu(tf.cast(x, tf.float32))

        adjustment_factor = 0.0
        for m in range(-self.m_range, self.m_range + 1):
            shift = m * self.h
            left_shift = g(-shift)
            right_shift = g(shift)
            adjustment_factor += current_loss * (left_shift - right_shift)

        adjustment_factor *= (self.h / 2)
        adjustment_factor = np.clip(adjustment_factor.numpy(), -self.max_adjustment, self.max_adjustment)
        return adjustment_factor

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}

        optimizer = self.model.optimizer
        try:
            if isinstance(optimizer.learning_rate, tf.Variable):
                current_lr = tf.keras.backend.get_value(optimizer.learning_rate)
            elif hasattr(optimizer.learning_rate, "numpy"):
                current_lr = optimizer.learning_rate.numpy()
            else:
                current_lr = optimizer.learning_rate
        except Exception as e:
            if self.verbose > 0:
                print(f"Epoch {epoch+1}: Unable to retrieve learning rate. Error: {e}")
            return

        adjustment_factor = self.compute_adjustment_factor(logs)
        self.loss_history.append(logs.get("loss", 1.0))
        if len(self.loss_history) > 3:
            recent_losses = np.array(self.loss_history[-3:])
            if np.all(recent_losses[1:] > recent_losses[:-1]):
                adjustment_factor *= 0.5

        if epoch < self.warmup_epochs:
            new_lr = current_lr * (1 + 0.2 * (epoch / self.warmup_epochs))
        else:
            new_lr = current_lr * self.decay_rate + adjustment_factor

        random_adjustment = 1 + rng.uniform(-self.random_factor, self.random_factor)
        new_lr *= random_adjustment
        new_lr = max(self.min_lr, new_lr)

        try:
            if isinstance(optimizer.learning_rate, tf.Variable):
                tf.keras.backend.set_value(optimizer.learning_rate, new_lr)
            else:
                optimizer.learning_rate = new_lr
        except Exception as e:
            if self.verbose > 0:
                print(f"Epoch {epoch+1}: Unable to set learning rate. Error: {e}")
            return

        if self.verbose > 0:
            print(f"Epoch {epoch+1}: Adjusted LR to {new_lr:.6f}")

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

# Define the LeNet-5 model
model = Sequential([
    # First convolutional layer (6 filters, 5x5 kernel, tanh activation)
    Conv2D(filters=6, kernel_size=(5, 5), activation='tanh', input_shape=(28, 28, 3), padding='same'),
    AveragePooling2D(pool_size=(2, 2)),

    # Second convolutional layer (16 filters, 5x5 kernel, tanh activation)
    Conv2D(filters=16, kernel_size=(5, 5), activation='tanh'),
    AveragePooling2D(pool_size=(2, 2)),

    # Flatten the feature maps
    Flatten(),

    # First fully connected layer (120 units, tanh activation)
    Dense(120, activation='tanh'),

    # Second fully connected layer (84 units, tanh activation)
    Dense(84, activation='tanh'),

    # Output layer (7 units, softmax activation for 7 classes)
    Dense(7, activation='softmax')
])


# Summary of the model
model.summary()

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Initial learning rate
    metrics=['accuracy']
)

custom_lr_scheduler = CustomLearningRateScheduler(h=1.0, m_range=2, verbose=1)

In [ ]:
import time
start_train_time = time.time()

history = model.fit(
    X_train, Y_train,
    validation_split=0.2,
    batch_size=128,
    epochs=20,
    callbacks=[custom_lr_scheduler]
)

train_time = time.time() - start_train_time
print(f" training time: {train_time:.2f} seconds")

In [ ]:
train_score = model.evaluate(X_train, Y_train, verbose= 1)
test_score = model.evaluate(X_test, Y_test, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
plt.figure(figsize=(12, 5))

# Loss Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Accuracy Plot
if 'accuracy' in history.history:  # For models using accuracy metric
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Epochs')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Define the baseline model
model1 = Sequential([
    # First convolutional layer (6 filters, 5x5 kernel, tanh activation)
    Conv2D(filters=6, kernel_size=(5, 5), activation='tanh', input_shape=(28, 28, 3), padding='same'),
    AveragePooling2D(pool_size=(2, 2)),

    # Second convolutional layer (16 filters, 5x5 kernel, tanh activation)
    Conv2D(filters=16, kernel_size=(5, 5), activation='tanh'),
    AveragePooling2D(pool_size=(2, 2)),

    # Flatten the feature maps
    Flatten(),

    # First fully connected layer (120 units, tanh activation)
    Dense(120, activation='tanh'),

    # Second fully connected layer (84 units, tanh activation)
    Dense(84, activation='tanh'),

    # Output layer (7 units, softmax activation for 7 classes)
    Dense(7, activation='softmax')
])

# Compile the model


# Instantiate the custom learning rate scheduler

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Triangular Learning Rate Policy
def triangular_lr(epoch, max_lr=0.00178, min_lr=0.00077, stepsize=10):
    """Implements the Triangular CLR policy."""
    cycle = np.floor(1 + epoch / (2 * stepsize))
    x = np.abs(epoch / stepsize - 2 * cycle + 1)
    lr = min_lr + (max_lr - min_lr) * max(0, (1 - x))
    return lr

# Define the LearningRateScheduler Callback
lr_callback = LearningRateScheduler(lambda epoch: triangular_lr(epoch, max_lr=0.00146, min_lr=0.00088, stepsize=10))


model1.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [ ]:
callback = ModelCheckpoint(
    filepath='best_model.keras',  # Changed to .keras extension
    monitor='val_accuracy',       # Monitoring validation accuracy
    verbose=1
)

In [ ]:
start_train_time = time.time()

history = model1.fit(
    X_train, Y_train,
    validation_split=0.2,
    batch_size=128,
    epochs=20,
    callbacks=[lr_callback]
)
train_time = time.time() - start_train_time
print(f" training time: {train_time:.2f} seconds")

In [ ]:
train_score = model1.evaluate(X_train, Y_train, verbose= 1)
test_score = model1.evaluate(X_test, Y_test, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
plt.figure(figsize=(12, 5))

# Loss Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Accuracy Plot
if 'accuracy' in history.history:  # For models using accuracy metric
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Epochs')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

# Define the LeNet-4 model
model2 = Sequential([
    # First convolutional layer (6 filters, 5x5 kernel, tanh activation)
    Conv2D(filters=4, kernel_size=(5, 5), activation='tanh', input_shape=(28, 28, 3), padding='same'),
    AveragePooling2D(pool_size=(2, 2)),

    # Second convolutional layer (16 filters, 5x5 kernel, tanh activation)
    Conv2D(filters=8, kernel_size=(5, 5), activation='tanh'),
    AveragePooling2D(pool_size=(2, 2)),

    # Flatten the feature maps
    Flatten(),



    # Output layer (7 units, softmax activation for 7 classes)
    Dense(7, activation='softmax')
])


# Summary of the model
model2.summary()

In [ ]:
model2.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Initial learning rate
    metrics=['accuracy']
)

In [ ]:
custom_lr_scheduler = CustomLearningRateScheduler(h=1.0, m_range=2, verbose=1)

In [ ]:
import time
start_train_time = time.time()

history = model2.fit(
    X_train, Y_train,
    validation_split=0.2,
    batch_size=128,
    epochs=20,
    callbacks=[custom_lr_scheduler]
)

train_time = time.time() - start_train_time
print(f" training time: {train_time:.2f} seconds")

In [ ]:
train_score = model2.evaluate(X_train, Y_train, verbose= 1)
test_score = model2.evaluate(X_test, Y_test, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
plt.figure(figsize=(12, 5))

# Loss Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Accuracy Plot
if 'accuracy' in history.history:  # For models using accuracy metric
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Epochs')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

# Define the LeNet-4 model
model3 = Sequential([
    # First convolutional layer (6 filters, 5x5 kernel, tanh activation)
    Conv2D(filters=4, kernel_size=(5, 5), activation='tanh', input_shape=(28, 28, 3), padding='same'),
    AveragePooling2D(pool_size=(2, 2)),

    # Second convolutional layer (16 filters, 5x5 kernel, tanh activation)
    Conv2D(filters=8, kernel_size=(5, 5), activation='tanh'),
    AveragePooling2D(pool_size=(2, 2)),

    # Flatten the feature maps
    Flatten(),



    # Output layer (7 units, softmax activation for 7 classes)
    Dense(7, activation='softmax')
])


# Summary of the model
model3.summary()

In [ ]:
model3.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [ ]:
callback = ModelCheckpoint(
    filepath='best_model.keras',  # Changed to .keras extension
    monitor='val_accuracy',       # Monitoring validation accuracy
    verbose=1
)

In [ ]:
train_score = model3.evaluate(X_train, Y_train, verbose= 1)
test_score = model3.evaluate(X_test, Y_test, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
plt.figure(figsize=(12, 5))

# Loss Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Accuracy Plot
if 'accuracy' in history.history:  # For models using accuracy metric
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Epochs')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

# Make predictions on training set (or validation set)
y_pred = model1.predict(X_train)
y_pred_classes = np.argmax(y_pred, axis=1)  # Get predicted class labels

# Check the shape of Y_train
if Y_train.ndim == 2:  # If Y_train is one-hot encoded
    y_true = np.argmax(Y_train, axis=1)  # Convert one-hot encoded labels to class indices
else:  # If Y_train is already in class index format
    y_true = Y_train  # Use it directly

# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plotting the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.arange(7), yticklabels=np.arange(7))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Plotting the loss and accuracy
plt.figure(figsize=(12, 5))

# Loss Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Accuracy Plot
if 'accuracy' in history.history:  # For models using accuracy metric
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Epochs')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

# Assuming model1 is already trained and X_train, Y_train are defined
# Make predictions on training set (or validation set)
y_pred = model1.predict(X_train)
y_pred_classes = np.argmax(y_pred, axis=1)  # Get predicted class labels

# Check the shape of Y_train
if Y_train.ndim == 2:  # If Y_train is one-hot encoded
    y_true = np.argmax(Y_train, axis=1)  # Convert one-hot encoded labels to class indices
else:  # If Y_train is already in class index format
    y_true = Y_train  # Use it directly

# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plotting the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.arange(7), yticklabels=np.arange(7))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Calculate precision, recall, and accuracy
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
accuracy = accuracy_score(y_true, y_pred_classes)

# Print the metrics
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Accuracy: {accuracy:.4f}')

# Optionally, print a detailed classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=[f'Class {i}' for i in range(7)]))

# Plotting the loss and accuracy
plt.figure(figsize=(12, 5))

# Loss Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Accuracy Plot
if 'accuracy' in history.history:  # For models using accuracy metric
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Epochs')
    plt.legend()

plt.tight_layout()
plt.show()